<a href="https://colab.research.google.com/github/maciejlipski/geo_llm/blob/main/geo_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Budowa Agenta Tekstowego dla Geologii/Geoinformatyki (RAG)
Kroki początkowe:
- wczytanie folderu 'pdf' zawierającego wybrane artykuły
- wczytanie pliku '.env' zawierającego klucz API OpenAI pod nazwą OPENAI_API_KEY

## 1. Instalacja bibliotek

In [ ]:
!pip install openai langchain faiss-cpu PyMuPDF sentence-transformers dotenv


## 2. Konfiguracja OpenAI

In [ ]:
import dotenv
import os
from openai import OpenAI

dotenv.load_dotenv('./.env')
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


## 3. Wczytanie dokumentów

In [ ]:
import os
import fitz

def load_pdfs(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            doc = fitz.open(os.path.join(folder_path, filename))
            full_text = "\n".join([page.get_text() for page in doc])
            texts.append(full_text)
    return texts

pdf_texts = load_pdfs("pdf")
print(f"Wczytano {len(pdf_texts)} dokumentów.")


## 4. Chunkowanie tekstów

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = []

for text in pdf_texts:
    split = text_splitter.create_documents([text])
    chunks.extend(split)

print(f"Liczba fragmentów po podziale: {len(chunks)}")


## 5. Tworzenie embeddingów i bazy FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

docs = [chunk.page_content for chunk in chunks]
embeddings = model.encode(docs)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


## 6. Funkcja retrieval (wyszukiwanie kontekstu)

In [ ]:
def retrieve_context(question, k=4):
    query_embedding = model.encode([question])
    _, indices = index.search(np.array(query_embedding), k)
    return "\n\n".join([docs[i] for i in indices[0]])


## 7. Funkcja zapytania do GPT-4

In [ ]:
def ask_gpt4(question, context):
    prompt = f"""
Odpowiedz na pytanie na podstawie poniższego kontekstu.

Kontekst:
{context}

Pytanie: {question}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()


## 8. Logowanie interakcji do pliku

In [ ]:
import csv

def log_interaction(question, answer, path="log.csv"):
    with open(path, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([question, answer])


##  9. Przykład działania agenta

In [ ]:
question = "Jakie metody stosuje się w geotechnice miejskiej do badania gruntu?"

context = retrieve_context(question)
answer = ask_gpt4(question, context)
print("📌 Odpowiedź GPT-4:\n", answer)

log_interaction(question, answer)


📌 Odpowiedź GPT-4:
 W geotechnice miejskiej do badania gruntu stosuje się różne metody sondowania, w tym sondowania statyczne i dynamiczne. Jedną z zalecanych metod jest sonda wkręcana, określana również jako Viktsond (Swedish Weight Sounding), która została opracowana w Szwecji. Metody te pozwalają na określenie właściwości gruntów, ich stanu oraz ściśliwości, co jest istotne dla celów dokumentowania geologiczno-inżynierskiego.

## 10. Intefejs z Inputem

In [ ]:
while True:
    user_question = input("\nZadaj pytanie (lub wpisz 'exit'): ")
    if user_question.lower() == 'exit':
        break
    context = retrieve_context(user_question)
    answer = ask_gpt4(user_question, context)
    print("\n🔹 Odpowiedź:\n", answer)
    log_interaction(user_question, answer)


Zadaj pytanie (lub wpisz 'exit'): Jakie są czynniki i kryteria wyboru właściwego kierunku rekultywacji?

🔹 Odpowiedź:
 Czynniki wyboru właściwego kierunku rekultywacji obejmują trzy grupy:

1. Koszty rekultywacji i zagospodarowania terenu poeksploatacyjnego.
2. Koszty utrzymania zagospodarowanego obszaru i obiektów.
3. Opłacalność inwestycji i okres zwrotu nakładów w przypadku przedsięwzięć komercyjnych.

Kryteria wyboru kierunku rekultywacji opierają się głównie na rachunku ekonomicznym, chyba że inne czynniki, takie jak kulturowe, okażą się istotniejsze. Ważne jest również uwzględnienie charakterystyki obiektu pod względem różnych czynników, co pozwala na określenie optymalnych funkcji dla obszarów poddawanych działaniom naprawczym.

Zadaj pytanie (lub wpisz 'exit'): exit